# Intraday Trading Strategies Demo

This notebook demonstrates the usage of the new intraday trading strategies:
- Morning Momentum
- Opening Range Breakout
- VWAP Mean Reversion
- Mean Reversion Intraday
- Sector Momentum

And shows how to:
1. Run strategies on historical data
2. Backtest strategies
3. Visualize results
4. Use the news-based forward tester

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from strategies import get_strategy, list_strategies
from app.backtest import BacktestEngine, BacktestConfig
from app.data_fetcher import DataFetcher
from news_forward_tester import NewsForwardTester

%matplotlib inline

## 1. List Available Strategies

In [ ]:
strategies = list_strategies()
print(f"Total strategies available: {len(strategies)}\n")

for strategy in strategies:
    print(f"ID: {strategy['id']}")
    print(f"Name: {strategy['name']}")
    print(f"Description: {strategy['description']}")
    print(f"Parameters: {strategy['parameters']}")
    print("-" * 50)

## 2. Fetch Intraday Data

In [ ]:
# Initialize data fetcher
data_fetcher = DataFetcher()

# Fetch 1-minute bars for a stock (e.g., AAPL)
symbol = 'AAPL'
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')

print(f"Fetching {symbol} data from {start_date} to {end_date}")

try:
    df = data_fetcher.get_bars(symbol, start_date, end_date, '5Min')
    print(f"Fetched {len(df)} bars")
    print(df.head())
except Exception as e:
    print(f"Error fetching data: {e}")
    # Create sample data for demo
    print("Creating sample data for demonstration...")
    dates = pd.date_range(start=start_date, end=end_date, freq='5Min')
    df = pd.DataFrame({
        'timestamp': dates,
        'open': 150 + np.random.randn(len(dates)).cumsum() * 0.5,
        'high': 151 + np.random.randn(len(dates)).cumsum() * 0.5,
        'low': 149 + np.random.randn(len(dates)).cumsum() * 0.5,
        'close': 150 + np.random.randn(len(dates)).cumsum() * 0.5,
        'volume': np.random.randint(1000, 10000, len(dates))
    })
    df['close'] = df[['open', 'high', 'low', 'close']].mean(axis=1)
    print(df.head())

## 3. Test Morning Momentum Strategy

In [ ]:
# Get strategy instance
strategy = get_strategy('morning_momentum', {
    'gap_threshold': 2.0,
    'rsi_period': 5,
    'rsi_max': 70,
    'volume_ratio_min': 2.0
})

print(f"Testing strategy: {strategy.name}")
print(f"Description: {strategy.description}")

# Generate signals
signals = []
for i in range(len(df)):
    signal = strategy.analyze(df, i)
    if signal.action != 'hold':
        signals.append(signal)
        print(f"Signal at {signal.timestamp}: {signal.action} - {signal.reason}")

print(f"\nTotal actionable signals: {len(signals)}")

## 4. Backtest Morning Momentum Strategy

In [ ]:
# Create backtest configuration
config = BacktestConfig(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    initial_capital=10000.0,
    commission=0.001,  # 0.1% commission
    strategy_id='morning_momentum',
    parameters={}
)

# Get fresh strategy instance
strategy = get_strategy('morning_momentum')

# Run backtest
engine = BacktestEngine(config, strategy)
result = engine.run(df)

# Display results
print("\n=== Backtest Results ===")
print(f"Strategy: {result.strategy_id}")
print(f"Symbol: {result.symbol}")
print(f"Period: {result.start_date} to {result.end_date}")
print(f"Initial Capital: ${result.initial_capital:,.2f}")
print(f"Final Capital: ${result.final_capital:,.2f}")
print(f"Total Return: ${result.total_return:,.2f} ({result.total_return_pct:.2f}%)")
print(f"Max Drawdown: {result.max_drawdown_pct:.2f}%")
print(f"Sharpe Ratio: {result.sharpe_ratio:.2f}")
print(f"Sortino Ratio: {result.sortino_ratio:.2f}")
print(f"Total Trades: {result.total_trades}")
print(f"Winning Trades: {result.winning_trades} ({result.win_rate:.1f}%)")
print(f"Avg Win: ${result.avg_win:.2f}")
print(f"Avg Loss: ${result.avg_loss:.2f}")
print(f"Profit Factor: {result.profit_factor:.2f}")

## 5. Visualize Equity Curve

In [ ]:
# Extract equity curve data
equity_df = pd.DataFrame(result.equity_curve)

if not equity_df.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Plot equity curve
    ax1.plot(equity_df.index, equity_df['equity'], label='Equity', linewidth=2)
    ax1.axhline(y=config.initial_capital, color='gray', linestyle='--', label='Initial Capital')
    ax1.set_title(f'{result.strategy_id} - Equity Curve')
    ax1.set_xlabel('Bar Index')
    ax1.set_ylabel('Equity ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot drawdown
    ax2.fill_between(equity_df.index, 0, -equity_df['drawdown'] * 100, 
                      alpha=0.3, color='red', label='Drawdown')
    ax2.set_title('Drawdown')
    ax2.set_xlabel('Bar Index')
    ax2.set_ylabel('Drawdown (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No equity curve data to plot")

## 6. Compare Multiple Strategies

In [ ]:
# List of intraday strategies to compare
strategy_ids = [
    'morning_momentum',
    'opening_range_breakout',
    'vwap_reversion',
    'mean_reversion_intraday',
    'sector_momentum'
]

results_comparison = []

for strategy_id in strategy_ids:
    try:
        strategy = get_strategy(strategy_id)
        config = BacktestConfig(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            initial_capital=10000.0,
            commission=0.001,
            strategy_id=strategy_id,
            parameters={}
        )
        
        engine = BacktestEngine(config, strategy)
        result = engine.run(df)
        
        results_comparison.append({
            'Strategy': strategy_id,
            'Return %': result.total_return_pct,
            'Sharpe': result.sharpe_ratio,
            'Max DD %': result.max_drawdown_pct,
            'Trades': result.total_trades,
            'Win Rate %': result.win_rate
        })
    except Exception as e:
        print(f"Error testing {strategy_id}: {e}")

# Display comparison
comparison_df = pd.DataFrame(results_comparison)
print("\n=== Strategy Comparison ===")
print(comparison_df.to_string(index=False))

## 7. News-Based Forward Testing

In [ ]:
# Initialize news forward tester
news_tester = NewsForwardTester()

# Sample news data
news_data = [
    {
        'symbol': 'AAPL',
        'headline': 'Apple announces strong quarterly earnings, beats expectations',
        'timestamp': datetime.now().isoformat()
    },
    {
        'symbol': 'AAPL',
        'headline': 'iPhone sales surge in Q4, driving record revenue',
        'timestamp': datetime.now().isoformat()
    },
    {
        'symbol': 'TSLA',
        'headline': 'Tesla stock drops on production concerns',
        'timestamp': datetime.now().isoformat()
    },
    {
        'symbol': 'GOOGL',
        'headline': 'Google unveils new AI features, stock rallies',
        'timestamp': datetime.now().isoformat()
    }
]

# Generate signals
signals = news_tester.generate_signals(news_data, top_n=3)

print("\n=== News-Based Signals ===")
for signal in signals:
    print(f"\nSymbol: {signal.symbol}")
    print(f"Action: {signal.action}")
    print(f"Sentiment: {signal.sentiment_label} (score: {signal.sentiment_score:.2f})")
    print(f"Confidence: {signal.confidence:.2f}")
    print(f"News Count: {signal.news_count}")
    print(f"Reason: {signal.reason}")

## 8. API Integration Example

In [ ]:
import requests

# Assuming the backend is running on localhost:8000
BASE_URL = 'http://localhost:8000'

# Example: List all strategies via API
response = requests.get(f'{BASE_URL}/strategy/list')
if response.status_code == 200:
    print("Available strategies via API:")
    strategies = response.json()['strategies']
    for s in strategies:
        print(f"  - {s['id']}: {s['name']}")
else:
    print(f"Error: {response.status_code}")
    print("Make sure the backend is running with: python main.py")

## Conclusion

This notebook demonstrated:
1. ✅ Five new intraday trading strategies
2. ✅ Backtesting framework with performance metrics
3. ✅ Strategy comparison
4. ✅ News-based sentiment analysis and forward testing
5. ✅ API integration examples

### Next Steps
- Fine-tune strategy parameters for your trading style
- Add real-time news feeds for forward testing
- Deploy strategies in paper trading mode
- Monitor performance and iterate